# Importing Libraries 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cmcdata/cmc-gain-dataset.xlsx
/kaggle/input/cmcdata2/cmc-gain-dataset.csv
/kaggle/input/cmc-gain-dataset/CMC_24h_Gainers_Starting_07_10_24 updated.csv


# Defining Filepaths

In [2]:
#Assumes that your csv file as a single row o'f defined header (starting point, first row)

cleanedFilepath = '/kaggle/working/cmc-gain-dataset-Clean.csv'
filepath = '/kaggle/input/cmcdata2/cmc-gain-dataset.csv'
csvFile = pd.read_csv(filepath)
print(csvFile.dtypes)#prints datatypes for each col
print(type(csvFile))#prints <class 'pandas.core.frame.DataFrame'>


cmc-gain-dataset.csv      int64
CMC_Rank                 object
Name                     object
Symbol                   object
Price($)                 object
24h%                    float64
24h_Vol($)               object
dtype: object
<class 'pandas.core.frame.DataFrame'>


# FillingNull Fxn

In [3]:
#To fill empty cells
def fillNullCells(workingCol):
    workingCol = pd.to_numeric(workingCol, errors='coerce')
    if pd.isna(workingCol.iloc[0]):#Probably first cell in col is empty
        colMedian = workingCol.median()
        workingCol.iloc[0] = colMedian#interpolate doesnt fill the first cell id its empty so need to fill manually
    workingCol.fillna(workingCol.interpolate(), inplace=True)#Using interpolate as it is best fit for time series data
    #print(workingCol)
    return workingCol


# Mapping Fxn

In [4]:
#To map each nonnumerical elemt to a num
def mapElmtToNum(workingCol):
    nonNum = []
    for idx, elmt in enumerate(workingCol):#itrs over each elmt in a single col 
        if pd.isna(elmt): #skips an empty cell
            continue
        try: #Try to get if its a num
            floatelmt = float(elmt)
        except: #Then value is str or unknown, i.e mix of chars
            nonNum.append(elmt)
    nonNum = pd.unique(pd.Series(nonNum))
    startNum = pd.to_numeric(workingCol, errors='coerce').max()+1#Trying to get max num in the col, turning all num to float and nonnum to nan
    mapped = {val: idx for idx, val in enumerate(nonNum, start=int(startNum))}
    workingCol = workingCol.map(lambda elmt: mapped.get(elmt, elmt))
    workingCol = pd.to_numeric(workingCol, errors='coerce')
    workingCol = fillNullCells(workingCol)
    return workingCol


# CleaningMixed Fxn

In [5]:
#To clean a num col mixed with other chars
def cleanObjNumericalCol(workingCol):
    numUniks = workingCol.nunique()
    numValues = sum(workingCol.isna() == False)
    if numUniks >= 0.6 * numValues:#To check if the col is not categorical i.e. not repeated values
        for idx, elmt in enumerate(workingCol):#itrs over each elmt in a single col 
            if pd.isna(elmt): #skips an empty cell
                continue
            elmt = re.sub(r'[^a-zA-Z0-9.]+', '', elmt)
        #    try:
        #        floatelmt = float(elmt)
        #    except:
        #        csvFile.at[idx, col] = np.nan
        workingCol = pd.to_numeric(workingCol, errors='coerce')
        workingCol = fillNullCells(workingCol)
        return workingCol
    else:
        return None


# TurnAlphabetsToLowercase Fxn

In [6]:
#To turn alphabets to lowercase
def turnAlphaLower(workingCol):
    for idx, elmt in enumerate(workingCol):#itrs over each elmt in a single col 
        if pd.isna(elmt): #skips an empty cell
            continue
        try: #To avoid float * isalpha * str * strip errors when there are empty cells in the col
            #elmt = re.sub('[^a-zA-Z0-9]+', '', str(elmt))#remove all whitespaces and special characters that negates alphabets or numbers from each elmt, no need for 'str'
            elmt = re.sub(r'[^a-zA-Z0-9]+', '', elmt)
            try: #To catch errors that will come from values other than numbers
                floatelmt = float(elmt)
            except: #Then value is str or unknown, i.e mix of chars
                if not elmt.isalpha() and  not elmt.isalnum():#If value tossed here is not an alphabet and not alphanumeric
                    csvFile.at[idx, col] = np.nan
                else:
                    csvFile.at[idx, col] = elmt.lower() #return the elmt in lowercase without spaces 
        except:
            continue #skip the empty cell
    return workingCol


# IsCategorical Fxn

In [7]:
#To determine for categorical col
def isCategorical(workingCol):
    numValues = sum(workingCol.isna() == False)#Total num of vals in the col
    numUniks = workingCol.nunique()#Total num of unique vals
    numTwiceUniques = sum(workingCol.value_counts() >= 2)#Total num of vals that reapeat atb least twice
    #print(workingCol.value_counts()) # To get a view of the freq of each val

    #Using max 60% should be unique threshold
    #isCat = (numUniks / numValues) #The ratio of unique vals to total vals
    #print(isCat)
    #if isCat <= 0.6:
        #Turn to category
    #else:
        #csvFile.drop(workingCol)#Means its not categorical so delete col
    #Using 60% of unique should repeat at least twice threshold
    isCat2 = (0.6 * numUniks)#preferred ratio
    #print(isCat2)
    if isCat2 <= numTwiceUniques:
        #Turn to categorical
        workingCol = mapElmtToNum(workingCol)
        return workingCol
    
    else:
        return None #This col is not cleaned



# Count Values Fxn

In [8]:
#To count each elmt in each col
def countObjsCol(workingCol):
    strCount = 0
    alphanumericCount = 0
    numCount = 0
    unknownCount = 0
    #for elmt in csvFile[col]:#itrs over each elmt in a single col
    
    for idx, elmt in enumerate(workingCol):#itrs over each elmt in a single col 
        if pd.isna(elmt): #skips an empty cell
            continue
        try: #To avoid float * isalpha * str * strip errors when there are empty cells in the col
            elmt = re.sub(r'[,\s]+', '', elmt)#remove all whitespaces and commas ONLY from each elmt (need to know unknowns)
            if elmt.isalpha():#Checks if each elmt is an alphabet
                strCount += 1
                #print(elmt)#prints the alphabetical elmt
            
            else:
                try: #To catch errors that will come from values other than numbers
                    floatnum = float(elmt)
                    numCount += 1
                except:
                    if elmt.isalnum():#For alphanum chars
                        alphanumericCount += 1 
                    else:
                        unknownCount += 1 #Then value is unknown, i.e mix of chars 
                    continue #skip the value
                
        except:
            continue #skip the empty cel
    return strCount, alphanumericCount, numCount, unknownCount


# Main Program

In [9]:

#Main Operation
for col in csvFile.columns:#itrs over all cols at once, but a single col with index def
    #pass
    #print(csvFile[col]) #Prints the current col
    workingCol = csvFile[col]

    if csvFile[col].dtype == int: #Its perfect so skip
        continue

    elif csvFile[col].dtype == float: #Might need fixing
        csvFile[col] = fillNullCells(csvFile[col])
        #print(csvFile[col])
        

    elif workingCol.dtype == object: #Checks for each cols first
        strCount, alphanumericCount, numCount, unknownCount = countObjsCol(workingCol)
        values = strCount + alphanumericCount + numCount
        
        #print(col)
        #print('String: ', strCount)
        #print('Alphanumeric: ', alphanumericCount)
        #print('Nums: ', numCount)
        #print('Unknown: ', unknownCount)
        
        if numCount >= (0.6 * values):#Incase a numerical col is mixed with few alphabets
            if cleanObjNumericalCol(csvFile[col]) is None:
                csvFile.drop(col, axis=1, inplace=True)
            else:
                csvFile[col] = turnAlphaLower(csvFile[col])
                csvFile[col] = cleanObjNumericalCol(csvFile[col])#Num col with some empty cells
                csvFile[col] = isCategorical(csvFile[col])
                #print(csvFile[col])

        #This works well for cols with more alpha chars than the others
        elif (strCount + alphanumericCount + numCount) >= (0.8 * sum(workingCol.isna() == False)): #This ratio means cols is likely mixed up with invalid data
            #To determine for Categorical col and then map
            if isCategorical(csvFile[col]) is None:
                csvFile.drop(col, axis=1, inplace=True)#Means its not categorical so delete col
            else:
                csvFile[col] = turnAlphaLower(csvFile[col])      
                csvFile[col] = isCategorical(csvFile[col])
                #print(csvFile[col])


        else:
            #print(workingCol)
            csvFile.drop(col, axis=1, inplace=True)#Means its not categorical so delete col
            
    else:
        csvFile.drop(col, axis=1, inplace=True)#Del any other col

print(csvFile.dtypes)

csvFile.to_csv(cleanedFilepath, index=False)
print('Clean Data Exported Successfully')


cmc-gain-dataset.csv      int64
Price($)                 object
24h%                    float64
24h_Vol($)               object
dtype: object
Clean Data Exported Successfully
